In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from keras.layers import LSTM
from keras import optimizers as opt

import mbspbs10pc.model as mbs_model
from mbspbs10pc import read_activations, utils
reload(mbs_model)
reload(read_activations)
reload(utils);

# Load data

In [ ]:
labelsfile = '../../tmp/1_METONLY_vs_METX/matched_CEM_table.csv'
datafile = '../../tmp/item_days_raw_data_.pkl'
dataset = utils.load_data_labels(datafile, labelsfile)
padded_mbs_seq, padded_timestamp_seq, _ = utils.tokenize(dataset)
maxlen = padded_mbs_seq.shape[1]

# Split in training, validation, test sets
tr_set, v_set, ts_set = utils.train_validation_test_split(
    [padded_mbs_seq, padded_timestamp_seq], dataset['Class'],
    test_size=0.4, validation_size=0.1,
    verbose=False)

In [ ]:
# x = tr_set[0][0]
# t = tr_set[0][1].squeeze()

# Load model

In [ ]:
model = mbs_model.build_model(mbs_input_shape=(maxlen,),
                              timestamp_input_shape=(maxlen, 1),
                              vocabulary_size=2774,
                              embedding_size=50,
                              recurrent_units=64,
                              dense_units=64,
                              bidirectional=True,
                              LSTMLayer=LSTM)

model.load_weights('../../tmp/auc_863/output_weights.h5')

# Visualize the attention weights

In [ ]:
a = read_activations.get_activations(model, tr_set[0], print_shape_only=True, layer_name='tsg_attention')[0]
c = read_activations.get_activations(model, tr_set[0], print_shape_only=True, layer_name='contribution')[0]

In [ ]:
avg_a = np.mean(a, axis=-1).squeeze()
avg_c = np.mean(c, axis=-1).squeeze()
print(avg_a.shape)
print(avg_a.shape)


In [ ]:
xaxis = np.arange(avg_a.shape[1])

plt.figure(figsize=(18,4))
for i in np.random.randint(0, avg_a.shape[0], size=10):
    plt.plot(xaxis, avg_a[i, :], '-')
#     plt.plot(xaxis, avg_c[i, :], '-')
plt.xlim([0, 445])
plt.legend(loc='best')